In [1]:
import numpy as np
import gurobipy as grb

In [7]:
X=Y=10
B=1000
np.random.seed(7)
phi_x_y = np.random.uniform(size = (X,Y))
epsilon_b_x_y = np.random.normal(size= (B,X,Y))
epsilon_b_x_0 = np.random.normal(size= (B,X))
eta_x_b_y =  np.random.normal(size= (X,B,Y))
eta_0_b_y =  np.random.normal(size= (B,Y))

In [8]:
m = grb.Model()
u_b_x = m.addMVar((B,X) , lb = -grb.GRB.INFINITY)
v_b_y = m.addMVar((B,Y), lb = -grb.GRB.INFINITY)
U_x_y = m.addMVar((X,Y), lb = -grb.GRB.INFINITY)
V_x_y = m.addMVar((X,Y), lb = -grb.GRB.INFINITY)
m.setObjective(u_b_x.sum() + v_b_y.sum(), sense = grb.GRB.MINIMIZE)
mu_x_y = m.addConstr(U_x_y + V_x_y == phi_x_y )
mu_b_x_y = m.addConstr(u_b_x[:,:,None] >= U_x_y[None,:,:] + epsilon_b_x_y )
mu_b_x_0 = m.addConstr(u_b_x >= epsilon_b_x_0 )
mu_x_b_y = m.addConstr(v_b_y[None,:,:] >= V_x_y[:,None,:] + eta_x_b_y )
mu_0_b_y = m.addConstr(v_b_y >= eta_0_b_y)


In [9]:
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 220100 rows, 20200 columns and 420200 nonzeros
Model fingerprint: 0x2523a1fd
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-07, 5e+00]
Presolve removed 20100 rows and 100 columns
Presolve time: 1.49s
Presolved: 20100 rows, 220000 columns, 420000 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Ordering time: 0.06s

Barrier statistics:
 AA' NZ     : 2.000e+05
 Factor NZ  : 2.256e+05 (roughly 100 MB of memory)
 Factor Ops : 2.782e+06 (less than 1 second per iteration)
 Threads    : 3

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Comp

In [10]:
mu_x_y.pi

array([[ 61.,  65., 127., 135., 129., 110.,  80.,  53., 121.,  72.],
       [ 73.,  67.,  65., 107., 116.,  87., 125., 117.,  78., 107.],
       [ 60., 111., 102.,  61., 110., 104., 119.,  92.,  86., 111.],
       [118.,  69., 107.,  76.,  77.,  73., 135.,  78.,  92., 119.],
       [120., 107.,  55.,  72., 116.,  86.,  74., 135.,  88.,  90.],
       [113.,  94., 118.,  96.,  76.,  99.,  72.,  68., 104., 112.],
       [134.,  89.,  72., 102.,  90.,  97.,  87.,  94.,  99.,  72.],
       [113., 108.,  77.,  97.,  74.,  91.,  88., 101.,  85., 101.],
       [ 88., 116.,  88.,  86.,  83., 118.,  62.,  87., 129.,  74.],
       [ 71., 113., 124., 101.,  90.,  95.,  88., 108.,  59.,  93.]])

In [13]:
n_x,m_y = np.ones(X),np.ones(Y)
a_x,b_y = np.zeros(X),np.zeros(Y)
K_x_y = np.exp(phi_x_y / 2)
maxIter = 10000
eps = 0.2
tol = 1e-5
for i in range(maxIter):
    mu_x_y = K_x_y * np.exp( (a_x[:,None]+b_y[None,:])/2 )
    mu_x0 = np.exp(a_x)
    mu_0y = np.exp(b_y)
    der_x = mu_x0 + mu_x_y.sum( axis = 1) - n_x 
    der_y = mu_0y + mu_x_y.sum( axis = 0) - m_y
    if max(np.max(np.abs(der_x / n_x)),np.max(np.abs(der_y / m_y)))<tol:
        break
    a_x -= eps * der_x
    b_y -= eps * der_y

print(i)
print(a_x,b_y)

85
[-2.68997396 -2.70265814 -2.71749333 -2.60391803 -2.64979922 -2.82749363
 -2.58534587 -2.54181749 -2.56559241 -2.68801907] [-2.71625255 -2.6029179  -2.61474448 -2.56459328 -2.73891937 -2.7302576
 -2.54203306 -2.64431606 -2.69423228 -2.71698553]


In [17]:
A_x,B_y = np.ones(X),np.ones(Y)
maxIter = 1000
for i in range(maxIter):
    halfKB_x = K_x_y @ B_y / 2
    A_x = np.sqrt(halfKB_x*halfKB_x + n_x) - halfKB_x
    halfAK_y = A_x @ K_x_y / 2
    B_y = np.sqrt(halfAK_y*halfAK_y + m_y) - halfAK_y
    
a_x,b_y = 2*np.log(A_x), 2*np.log(B_y)
print(a_x,b_y)

[-2.69008341 -2.70276748 -2.71760258 -2.60402944 -2.64991112 -2.82759862
 -2.58545785 -2.54192916 -2.56570492 -2.68812738] [-2.71613822 -2.60281044 -2.61463802 -2.56448845 -2.73880475 -2.73014385
 -2.54192865 -2.64420594 -2.69412038 -2.7168726 ]
